# Q1

In [4]:
import pandas as pd

year = '2023'
month = '03'
url=f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year}-{month}.parquet"
march_data = pd.read_parquet(url)

In [5]:
march_data.shape

(72044, 20)

# Q2

In [6]:
from evidently.metrics import ColumnQuantileMetric
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.utils.data_preprocessing import create_data_definition
from evidently.base_metric import InputData

In [7]:
def idk(df):
    cm = ColumnQuantileMetric(column_name='fare_amount', quantile=0.5)
    column_mapping = ColumnMapping()
    current_data = df
    data_definition = create_data_definition(None, current_data, column_mapping)
    data = InputData(None, current_data, None, None, column_mapping, data_definition)
    return cm.calculate(data=data)

In [8]:
idk(march_data).current.value

13.5

# Q3


According to the Prefect Docs (https://docs.prefect.io/2.10.18/concepts/tasks/) it should be;

`@task(retries=2, retry_delay_seconds=5, name="calculate metrics")`

# Q4

In [10]:
pick_march_2023 = (march_data.lpep_pickup_datetime.dt.year == 2023) & \
    (march_data.lpep_pickup_datetime.dt.month == 3) & \
    (march_data.lpep_dropoff_datetime.dt.year == 2023) & \
    (march_data.lpep_dropoff_datetime.dt.month == 3)

march_data = march_data[pick_march_2023]
march_data

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-03-01 00:25:10,2023-03-01 00:35:47,N,1.0,82,196,1.0,2.36,13.50,1.0,0.5,0.00,0.00,NaN,1.0,16.00,2.0,1.0,0.00
1,2,2023-03-01 00:14:29,2023-03-01 00:25:04,N,1.0,7,7,1.0,0.78,-6.50,-1.0,-0.5,0.00,0.00,NaN,-1.0,-9.00,3.0,1.0,0.00
2,2,2023-03-01 00:14:29,2023-03-01 00:25:04,N,1.0,7,7,1.0,0.78,6.50,1.0,0.5,0.00,0.00,NaN,1.0,9.00,3.0,1.0,0.00
4,2,2023-03-01 00:54:03,2023-03-01 01:03:14,N,1.0,236,229,1.0,3.14,15.60,1.0,0.5,4.17,0.00,NaN,1.0,25.02,1.0,1.0,2.75
5,2,2023-03-01 01:00:09,2023-03-01 01:14:37,N,1.0,75,235,1.0,5.69,23.30,1.0,0.5,4.00,0.00,NaN,1.0,29.80,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72038,2,2023-03-31 23:21:00,2023-03-31 23:43:00,None,NaN,7,61,NaN,9.35,33.43,0.0,0.0,5.16,0.00,NaN,1.0,39.59,NaN,NaN,NaN
72039,2,2023-03-31 23:33:00,2023-03-31 23:45:00,None,NaN,25,144,NaN,2.93,16.46,0.0,0.0,4.04,0.00,NaN,1.0,24.25,NaN,NaN,NaN
72040,2,2023-03-31 23:27:00,2023-03-31 23:53:00,None,NaN,36,236,NaN,8.49,34.66,0.0,0.0,7.68,0.00,NaN,1.0,46.09,NaN,NaN,NaN
72041,2,2023-03-31 23:09:00,2023-03-31 23:40:00,None,NaN,42,225,NaN,12.27,41.79,0.0,0.0,7.40,6.55,NaN,1.0,56.74,NaN,NaN,NaN


In [12]:
q = []
for i in range(1, 32):
    daily_data = march_data[march_data.lpep_pickup_datetime.dt.day == i]
    q50 = idk(daily_data).current.value
    print(f'Day {i}: Fare Amount (Q50) = {q50}')
    q.append(q50)
    
max(q)

Day 1: Fare Amount (Q50) = 12.8
Day 2: Fare Amount (Q50) = 13.5
Day 3: Fare Amount (Q50) = 13.5
Day 4: Fare Amount (Q50) = 13.5
Day 5: Fare Amount (Q50) = 13.5
Day 6: Fare Amount (Q50) = 13.05
Day 7: Fare Amount (Q50) = 12.8
Day 8: Fare Amount (Q50) = 12.8
Day 9: Fare Amount (Q50) = 13.5
Day 10: Fare Amount (Q50) = 13.5
Day 11: Fare Amount (Q50) = 13.5
Day 12: Fare Amount (Q50) = 12.99
Day 13: Fare Amount (Q50) = 12.8
Day 14: Fare Amount (Q50) = 12.8
Day 15: Fare Amount (Q50) = 13.5
Day 16: Fare Amount (Q50) = 13.5
Day 17: Fare Amount (Q50) = 13.5
Day 18: Fare Amount (Q50) = 14.0
Day 19: Fare Amount (Q50) = 13.5
Day 20: Fare Amount (Q50) = 13.0
Day 21: Fare Amount (Q50) = 13.5
Day 22: Fare Amount (Q50) = 13.5
Day 23: Fare Amount (Q50) = 13.5
Day 24: Fare Amount (Q50) = 13.5
Day 25: Fare Amount (Q50) = 13.5
Day 26: Fare Amount (Q50) = 13.5
Day 27: Fare Amount (Q50) = 12.8
Day 28: Fare Amount (Q50) = 13.5
Day 29: Fare Amount (Q50) = 13.5
Day 30: Fare Amount (Q50) = 13.51
Day 31: Fare Amo

14.0